In [ ]:

# source: https://ogb.stanford.edu/docs/linkprop/#ogbl-collab

from ogb.linkproppred import LinkPropPredDataset

dataset = LinkPropPredDataset(name = 'ogbl-collab')

split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
graph = dataset[0]



In [ ]:
import numpy as np




In [ ]:
node_feats = graph['node_feat'] # (235868, 128)
edge_year = graph['edge_year']
edge_weight = graph['edge_weight'].reshape(-1,) # (2358104, )
edge = graph['edge_index'].T # (2358104, 2)
neg_edge = np.concatenate([valid_edge['edge_neg'], test_edge['edge_neg']])
node_list = np.ones((1, graph['num_nodes']))
edge_list = np.ones((1, edge.shape[0]))
PositiveEdge_id = np.arange(0, edge.shape[0])
NegativeEdge_id = np.arange(edge.shape[0], edge.shape[0] + neg_edge.shape[0])

data = {
    "node_feats": node_feats,
    "edge_year": edge_year,
    "edge_weight":edge_weight,
    "edge": edge,
    "neg_edge": neg_edge,
    "node_list": node_list,
    "edge_list": edge_list,
    "PositiveEdge_id": PositiveEdge_id,
    "NegativeEdge_id": NegativeEdge_id
}

In [ ]:
np.savez_compressed("ogbl-collab.npz", **data)



In [ ]:
collab = np.load("ogbl-collab.npz", allow_pickle=True)
collab.files

In [ ]:
task_data = {
    # use edge_id to index val_neg and test_neg
    "val": np.arange(edge.shape[0], edge.shape[0] + valid_edge['edge_neg'].shape[0]),
    "test": np.arange(edge.shape[0]+valid_edge['edge_neg'].shape[0], edge.shape[0]+valid_edge['edge_neg'].shape[0] + test_edge['edge_neg'].shape[0]),
    "train_time_window": (1963, 2017),
    "valid_time_window": (2018, 2018),
    "test_time_window": (2019, 2019)
}
np.savez_compressed("ogbl-collab_task_prestore_neg.npz", **task_data)




In [ ]:
task_data_no_pre_stored_neg_edge = {
    "train_time_window": (1963, 2017), # window of (edge_year in train)
    "valid_time_window": (2018, 2018),
    "test_time_window": (2019, 2019)
}
np.savez_compressed("ogbl-collab_task_runtime_sampling.npz", **task_data_no_pre_stored_neg_edge)

In [ ]:
# 1. pre-store neg - index
# 2. no pre-store - time